<a href="https://colab.research.google.com/github/yufengg/advent-of-code-2024/blob/master/AoC_2024_day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: python code download advent of code file

import requests
import os
from datetime import datetime

def download_aoc_input(year, day, session_cookie):
  """Downloads the input file for a specific Advent of Code day.

  Args:
    year: The year of the challenge (e.g., 2023).
    day: The day of the challenge (e.g., 1).
    session_cookie: Your session cookie from the Advent of Code website.
  """

  url = f"https://adventofcode.com/{year}/day/{day}/input"
  headers = {
      "User-Agent": "github.com/yourusername/yourrepo (or your email)",  # Replace with your info
      "Cookie": f"session={session_cookie}"
  }

  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Create a directory for the year if it doesn't exist
    os.makedirs(str(year), exist_ok=True)

    filename = os.path.join(str(year), f"day{day}.txt")
    with open(filename, "w") as file:
      file.write(response.text)
    print(f"Successfully downloaded input for year {year}, day {day} to {filename}")
    return filename

  except requests.exceptions.RequestException as e:
    print(f"Error downloading input: {e}")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")


# Example usage:  Replace with your actual values
year = 2024  #@param {type:"integer"}
day = 2 #@param {type:"integer"}
session_cookie = "53616c7465645f5fd6809f86933785dcec3c4f79b521758a796b0808898e32ff9a09d06dd15282b2a68340b7bf970ab66894fa0bf9271caa521bc5d5fc8df3b9" #@param {type:"string"}

input_filename = download_aoc_input(year, day, session_cookie)

Successfully downloaded input for year 2024, day 2 to 2024/day2.txt


In [2]:
import jax

In [3]:
import jax.numpy as jnp

In [4]:

def read_input(filename, parser=None):
  with open(filename, 'r') as file:
    lines = file.read().splitlines()
    if parser:
      lines = list(map(parser, lines))
  return lines


In [7]:
input_filename = f'{year}/day{day}.txt'
def parser(line):
  return list(map(int, line.split()))

input_lines = read_input(input_filename, parser)

For each line, figure out if things are all inc/dec, *OR*
if the change is `x in (1,2,3)`

In [81]:
def calc_diffs(report):
  # eg [75, 76, 77, 80, 82, 85, 84]
  # print(report)
  diffs = []
  for x in range(len(report)-1):
    diff = report[x] - report[x+1]
    diffs.append(diff)
  diffs = jnp.array(diffs)
  return diffs

In [66]:
diffs = [calc_diffs(report) for report in input_lines]
# diffs

[75, 76, 77, 80, 82, 85, 84]
[49, 52, 53, 55, 58, 59, 61, 61]
[54, 57, 60, 62, 66]
[4, 6, 8, 10, 11, 14, 19]
[82, 85, 86, 83, 85, 87, 89]
[85, 86, 87, 86, 89, 88]
[69, 72, 73, 76, 77, 74, 75, 75]
[4, 6, 4, 5, 8, 11, 15]
[6, 9, 10, 12, 15, 16, 15, 20]
[41, 43, 43, 45, 48, 51]
[48, 51, 53, 53, 56, 58, 59, 58]
[67, 69, 71, 74, 74, 77, 77]
[82, 83, 85, 85, 89]
[21, 24, 26, 26, 29, 31, 36]
[4, 7, 10, 11, 15, 18, 20, 21]
[56, 58, 60, 63, 65, 69, 70, 68]
[24, 26, 30, 32, 33, 35, 35]
[12, 13, 15, 17, 20, 24, 25, 29]
[52, 53, 54, 57, 58, 59, 63, 68]
[43, 46, 48, 55, 58, 59]
[15, 16, 23, 26, 27, 28, 26]
[49, 50, 53, 56, 59, 64, 64]
[56, 58, 59, 65, 69]
[3, 5, 6, 8, 13, 18]
[45, 44, 46, 47, 50, 52, 54]
[37, 35, 36, 37, 34]
[53, 50, 51, 53, 54, 56, 56]
[80, 78, 79, 80, 81, 83, 86, 90]
[32, 31, 34, 37, 40, 45]
[76, 75, 77, 79, 77, 80]
[26, 25, 22, 24, 23]
[27, 26, 23, 24, 25, 25]
[96, 93, 96, 93, 97]
[37, 34, 35, 36, 39, 36, 38, 45]
[68, 65, 66, 67, 67, 70, 72]
[41, 39, 42, 42, 43, 40]
[88, 86, 89,

In [67]:
def all_one_sign(diffs):
  if diffs[0] > 0:
    return jnp.all(diffs > 0) and jnp.all(diffs <=3)
  if diffs[0] < 0:
    return jnp.all(diffs < 0) and jnp.all(diffs >=-3)

  return False # if the first val == 0

In [68]:
def rules(diffs):
  return all_one_sign(diffs)

In [70]:
scores = [rules(diff) for diff in diffs]
sum(scores)

Array(479, dtype=int32, weak_type=True)

# Part 2
Go through each one that didn't work, and try to remove 1 element, 1 by 1, and recheck each, until success, or no more elements.

In [82]:
new_true = 0
for i, outcome in enumerate(scores):
  if outcome == True:
    continue
  report = input_lines[i]
  for e in range(len(report)):
    modified_report = report[0:e] + report[e+1:]
    mod_diffs = calc_diffs(modified_report)
    if rules(mod_diffs):
      print(modified_report)
      new_true += 1
      break

new_true

[75, 76, 77, 80, 82, 84]
[49, 52, 53, 55, 58, 59, 61]
[54, 57, 60, 62]
[4, 6, 8, 10, 11, 14]
[41, 43, 45, 48, 51]
[44, 46, 47, 50, 52, 54]
[26, 25, 24, 23]
[80, 83, 86, 87]
[15, 16, 18, 20, 21, 23, 26]
[51, 53, 55, 56]
[63, 61, 58, 56, 54, 53]
[52, 49, 46, 45]
[74, 72, 69, 68, 67]
[80, 77, 74, 73, 72]
[77, 76, 73, 70, 68, 67]
[17, 14, 12, 10, 8, 6]
[84, 81, 80, 78]
[29, 26, 24, 23, 22, 19]
[77, 76, 74, 72, 71, 69, 66]
[22, 21, 19, 18, 15]
[46, 43, 41, 38, 37, 34]
[49, 51, 52, 54]
[93, 94, 95, 96]
[23, 24, 26, 28, 29, 31, 34]
[83, 85, 87, 88, 89]
[66, 67, 68, 70]
[35, 36, 39, 40]
[10, 11, 13, 16, 19]
[91, 92, 95, 98]
[66, 68, 69, 72]
[53, 54, 56, 59, 60, 62]
[63, 66, 67, 68, 71, 74]
[32, 35, 38, 41, 44, 45]
[5, 7, 10, 12]
[50, 53, 55, 56, 58]
[60, 57, 56, 54, 53, 51]
[97, 94, 93, 92, 90, 88, 86]
[67, 64, 62, 61]
[74, 72, 71, 68, 65]
[97, 95, 92, 91, 90, 89]
[88, 85, 82, 80]
[9, 8, 5, 2]
[75, 72, 70, 69]
[28, 27, 26, 23]
[16, 14, 12, 10, 8, 7, 6]
[74, 71, 70, 68, 65]
[37, 34, 31, 30]
[52

52

In [78]:
a = [0,1,2,3,4,5]
for e in range(len(a)):
  print(a[0:e] + a[e+1:])

[1, 2, 3, 4, 5]
[0, 2, 3, 4, 5]
[0, 1, 3, 4, 5]
[0, 1, 2, 4, 5]
[0, 1, 2, 3, 5]
[0, 1, 2, 3, 4]
